# Loop all crop types

(and save CSV)

keep: id_17

| new columns: | | | | | | | | |
| :- | :- | :- | :- | :- | :- | :- | :- | :- |
| | dist_18 | id_18-0 | id_18-1 | id_18-2 | dist_19 | id_19-0 | id_19-1 | id_19-2 |













In [1]:
%%time

# Import modules
import geopandas as gpd
import pandas as pd
import numpy as np
from sklearn.neighbors import BallTree

# Import shape files
clean_17 = gpd.read_file(".\\poljine_SHP\\ZV2017_d96tm_clean_ids.shp")
clean_18 = gpd.read_file(".\\poljine_SHP\\ZV2018_d96tm_clean_ids.shp")
clean_19 = gpd.read_file(".\\poljine_SHP\\KMRS2019_d96tm_clean_ids.shp")

Wall time: 3min 2s


In [11]:
%%time

# (0) Select the year for processing & and set number of closest neighbours
year = 2018
no_near = 3

# (1) Create Df copy for output and
output = clean_17.copy()

# (2) add new columns for that will be use for finding nearest neighbours
ktree_labels = ["dist0"] + [f"near{n}" for n in range(no_near)]
for lb in ktree_labels:
    output[lb] = None
    
# (3) Tidy (remove unneeded columns)
output = output.drop(columns=["RABA_ID", "POVR_GERK_"])
output = output.drop(output.loc[:, "OBRAZEC" : "REGIJA"].columns, axis=1)

output.head(2)

Wall time: 800 ms


,id_17,GERK_PID,POLJINA_ID,SIFRA_KMRS,geometry,dist0,near0,near1,near2
0,0,2537988,2104906,801,"POLYGON ((547508.347 86829.736, 547545.426 868...",None,None,None,None
1,1,2537988,2104908,005,"POLYGON ((547497.850 86785.076, 547578.847 867...",None,None,None,None


Now we need to filter the data. We are only interseted in the crop types that have at least 1000 instances for each year.

This cell is a little bigger, just so we can see which crop types were excluded:

In [12]:
%%time

threshold = 1000

print(f"Removing crop types with less than {threshold} occurances in 2017:\n--")

# Get statistics on KMRS values (crop type)
kmrs_list = clean_17.SIFRA_KMRS.value_counts()
# Only select values that have more than 1000 occurances 
k_list = kmrs_list[kmrs_list <= threshold].index.to_list()  # Excluded indices
kmrs_list = kmrs_list[kmrs_list > threshold].index.to_list()
print(f"{len(kmrs_list)} elements:\n {kmrs_list}")

for k in k_list:
    rem_17 = str(clean_17.SIFRA_KMRS.value_counts().loc[k])

    try:
        rem_18 = str(clean_18.SIFRA_KMRS.value_counts().loc[k])
    except:
        rem_18 = "0"

    try:
        rem_19 = str(clean_19.SIFRA_KMRS.value_counts().loc[k])
    except:
        rem_19 = "0"
    print(f"Removed {k} | {rem_17} instances in 2017 | {rem_18} instances in 2018 | {rem_19} instances in 2019")

# Check if all croptypes from kmrs_list are available in 2018 and 2019, remove if not
kmrs_18 = clean_18.SIFRA_KMRS.value_counts()
kmrs_18 = kmrs_18[kmrs_18 > threshold].index.to_list()
kmrs_list_a = kmrs_list.copy()  # Had to do this to not prevent skipping of elements in the list
for k in kmrs_list:
    if k not in kmrs_18:
        kmrs_list_a.remove(k)
        rem_17 = clean_17.SIFRA_KMRS.value_counts().loc[k]
        
        try:
            rem_18 = clean_18.SIFRA_KMRS.value_counts().loc[k]
        except:
            rem_18 = 0
        
        try:
            rem_19 = clean_19.SIFRA_KMRS.value_counts().loc[k]
        except:
            rem_19 = 0
        
        print(f"Removed {k} | {rem_17} instances in 2017 | {rem_18} instances in 2018 | {rem_19} instances in 2019")

print(f"{len(kmrs_list_a)} elements:\n {kmrs_list_a}")

kmrs_19 = clean_19.SIFRA_KMRS.value_counts()
kmrs_19 = kmrs_19[kmrs_19 > threshold].index.to_list()
kmrs_list_b = kmrs_list_a.copy()
for k in kmrs_list_a:
    if k not in kmrs_19:
        kmrs_list_b.remove(k)
        rem_17 = str(clean_17.SIFRA_KMRS.value_counts().loc[k])
        
        try:
            rem_18 = str(clean_18.SIFRA_KMRS.value_counts().loc[k])
        except:
            rem_18 = "0"
        
        try:
            rem_19 = str(clean_19.SIFRA_KMRS.value_counts().loc[k])
        except:
            rem_19 = "0"
        
        print(f"Removed {k} | {rem_17} instances in 2017 | {rem_18} instances in 2018 | {rem_19} instances in 2019")

print(f"{len(kmrs_list_b)} elements:\n {kmrs_list_b}")

kmrs_list = kmrs_list_b.copy()

# append 004 - ajda (see notes in the notebook below)
kmrs_list.append("004")

Removing crop types with less than 1000 occurances in 2017:
--
34 elements:
 ['204', '005', '801', '809', '006', '100', '699', '203', '201', '208', '807', '013', '112', '405', '207', '020', '814', '206', '004', '402', '222', '555', '611', '111', '008', '506', '030', '811', '802', '812', '009', '800', '026', '803']
Removed 808 | 881 instances in 2017 | 841 instances in 2018 | 653 instances in 2019
Removed 001 | 877 instances in 2017 | 578 instances in 2018 | 591 instances in 2019
Removed 033 | 854 instances in 2017 | 551 instances in 2018 | 469 instances in 2019
Removed 706 | 820 instances in 2017 | 936 instances in 2018 | 874 instances in 2019
Removed 900 | 661 instances in 2017 | 676 instances in 2018 | 660 instances in 2019
Removed 631 | 545 instances in 2017 | 618 instances in 2018 | 709 instances in 2019
Removed 027 | 450 instances in 2017 | 429 instances in 2018 | 478 instances in 2019
Removed 014 | 442 instances in 2017 | 378 instances in 2018 | 23 instances in 2019
Removed 621 |

In the end, we are left with 25 crop types as seen in from the final list above.

Excluded were:

| KMRS    | instances in 2017 | instances in 2018 | instances in 2019 | crop type |
| :--     | :-:  | :-:  | :-: | :--- |
| **803** | 1006 | 964  | 715 | pira (ozimna) |
| **112** | 7125 | 7280 | 33  | krmna ogrščica (jara) |
| ~~**004**~~ | ~~4009~~ | ~~4173~~ | ~~827~~ | ~~ajda~~ |
| **222** | 3262 | 3154 | 710 | inkarnatka |
| **555** | 2765 | 2549 | 0   | brez zahtevka |
| **111** | 2468 | 2595 | 46  | bela gorjušica |
| **506** | 2220 | 2331 | 0   | mešanica rastlin - naknadni posevek |
| **811** | 1922 | 2281 | 80  | mešanice žit (ozimna) |
| **812** | 1613 | 1578 | 69  | krmna ogrščica (ozimna) |

> `004` was added back, because it has been already used in the Susa project (it got accidentaly filtered out becuse it had less tha 1000 samples in the year 2019)

We have now built a list of all valid crop-types.

Next, we will search for nearest neighbours between two shapefiles. It includes the following steps:

1) FILTER THE DATA (have to do this for each KMRS type separately)

2) BUILD SPATIAL INDEX (based on centroids)

3) FIND CORESPONDING VALUES (and fill the 2017-20xx table)


In [13]:
%%time

for kmrs in kmrs_list:
    """(1) FILTER"""
    # Filter dataframe to include only this crop type
    filter_17 = clean_17["SIFRA_KMRS"] == kmrs
    data_17 = clean_17[filter_17]
    # Make sure you have the right year for the right data frame
    if year == 2018:
        filter_18 = clean_18["SIFRA_KMRS"] == kmrs
        data_18 = clean_18[filter_18]
    elif year == 2019:
        filter_18 = clean_19["SIFRA_KMRS"] == kmrs
        data_18 = clean_19[filter_18]
    else:
        print(f"Wrong year selected: {year}")
    
    """(2) BUILD K-TREE"""
    # Calculate centroid and parse coordinates
    src_points = np.array(data_17.centroid.apply(lambda geom: (geom.x, geom.y)).to_list())
    candidates = np.array(data_18.centroid.apply(lambda geom: (geom.x, geom.y)).to_list())

    # Set this to 3 to find the 3 closest (parametrized, set above with inputs)
    k_neighbors = no_near

    # Create tree from the candidate points (make sure you use arithmetic distance)
    tree = BallTree(candidates, leaf_size=15)
    
    # Find closest points and distances
    distances, indices = tree.query(src_points, k=k_neighbors)
    
    """(3) POPULATE DATA FRAME WITH THE RESULTS"""
    # K-tree query returns indices in an array, which means we lost the indices from the original data frame (candidates)
    my_index = indices.flatten()
    # Find corresponding indices (make sure to have the correct year)
    id_year = f"id_{str(year)[-2:]}"  # either "id_18" or "id_19"
    new_ids = data_18[id_year].reset_index(drop=True).iloc[my_index].to_numpy()
    # Change back to original shape
    ids_18 = new_ids.reshape(indices.shape)
    
    # Populate the output data frame
    output.loc[filter_17, ktree_labels[0]] = distances[:, 0].tolist()
    output.loc[filter_17, ktree_labels[1:]] = ids_18.tolist()

Wall time: 1min 55s


In [14]:
%%time

# SAVE CSV
output.drop(columns=["geometry"]).to_csv(f".\\results_ids\\ZV2017_nearest_ids_{str(year)[-2:]}_ajda.csv", index=False)

Wall time: 3.7 s


In [15]:
%%time

# SAVE OUTPUT
output.to_file(f".\\results_ids\\ZV2017_nearest_ids_{str(year)[-2:]}_ajda.shp")

Wall time: 3min 34s
